### Solves the 2D Wave equation with MDM and position Verlet algorithm

In [1]:
import numpy as np
from core import *

In [2]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
from matplotlib import animation, rcParams
from IPython.display import HTML

In [3]:
rcParams['animation.html'] = 'html5'

In [4]:
# Spatial discretization

# Order of accuracy
k = 4

# Spatial resolution
m = n = 101

# West
a = -5
# East
b = 10
# South
c = -5
# North
d = 10

dx = (b - a) / m
dy = (d - c) / n

# 2D Staggered grid
xgrid = np.append(np.insert(np.arange(a + dx/2., b, dx), 0, a), b)
ygrid = np.append(np.insert(np.arange(c + dy/2., d, dy), 0, c), d)

X, Y = np.meshgrid(xgrid, ygrid, indexing='ij')

# Mimetic Operator (Laplacian)
L = lap2D(k, m, dx, n, dy)
# Dirichlet BC
L = L + robinBC2D(k, m, dx, n, dy, 1, 0)
I = interpol2D(m, n, 0.5, 0.5)
I2 = interpolD2D(m, n, 0.5, 0.5)

# Wave propagation speed
c = 100

# "Force" function
def F(x, c):
    return (c**2) * L @ x  # c^2 DivGrad x

# Simulation time
TIME = 0.3

# Temporal discretization based on CFL condition
# dt = h on Young's paper
dt = dx / (2 * c)

# Initial condition
# Initial position of particles
def ICU(x, y):
    return np.sin(np.pi * x) * np.sin(np.pi * y)\
           * (x > 2).astype(np.int) * (x < 3).astype(np.int)\
           * (y > 2).astype(np.int) * (y < 3).astype(np.int)
# Initial velocity of particles
def ICV(x, y):
    return np.zeros(2*m*n + m + n)

uold = ICU(X, Y)
vold = ICV(X, Y)
uold = np.reshape(uold, (m+2)*(n+2))

# From Peter Young's paper
theta = 1. / (2. - 2.**(1./3.))

# Premultiply I and I2
I = dt * I
I2 = 0.5 * dt * I2

Nt = int(TIME / dt)
solution = np.zeros((m+2, n+2, Nt))

# Time integration loop
for t in range(0, Nt):
    # Apply "position Verlet" algorithm (2nd-order in time)
    uold = uold + I2 @ vold
    vnew = vold + I @ F(uold, c)
    unew = uold + I2 @ vnew
    
    # Update
    uold = unew.copy()
    vold = vnew.copy()
    solution[..., t] = np.reshape(unew, (m+2, n+2))

/Users/nutrik/Documents/Science/scodes/Python/mole/core/div.py:53: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  D[i, j:j+k] = coeffs
/Users/nutrik/Documents/Science/scodes/Python/mole/core/div.py:73: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  A[i, 0:q] = coeffs
/Users/nutrik/Documents/Science/scodes/Python/mole/core/div.py:79: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  D[1:p+1, 0:q] = A
/Users/nutrik/Documents/Science/scodes/Python/mole/core/div.py:95: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  D[n_rows-p-1:n_rows-1, n_cols-q:n_cols] = A
/Users/nutrik/Documents/Science/scodes/Python/mole/core/div2D.py:27: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive

In [5]:
def animate_wave2d(solution):
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111, projection='3d')

    # Setting the axes properties
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlim3d([-1.0, 1.0])
    ax.set_zlabel('U(X,Y)')
    x, y = np.meshgrid(xgrid, ygrid)
    surface = [ax.plot_surface(x, y, solution[:, :, 0], color='0.75', rstride=1, cstride=1, vmin=-1, vmax=1)]
    plt.close()

    def update_plot(frame_number, data, surface):
        surface[0].remove()
        surface[0] = ax.plot_surface(x, y, data[:, :, frame_number], cmap="gnuplot2", vmin=-1, vmax=1)

    return animation.FuncAnimation(fig, update_plot, frames=Nt, fargs=(solution, surface), interval=50)

In [6]:
anim = animate_wave2d(solution)

In [7]:
anim